# Dataset description
Data for this project is obtained from kaggle. The link to the data is https://www.kaggle.com/code/youben/twitter-sentiment-analysis/data

In [1]:
#Import required library
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import re
import os
import shutil
import string
import matplotlib as plt
import pydot 
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from collections import Counter
from tensorflow.keras import losses

from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.preprocessing.sequence import  pad_sequences
import emoji

2022-07-14 18:56:13.524355: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-14 18:56:13.524383: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/bonface/miniconda3/envs/tensorflow/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


# Library description
## Shutil
The shutil in Python is a module used for  several functions mostly to to deal with  file operations  and their collections.It used to copy or  removal files.It resembkes OS module but the only difference is that the OS module lacks functions dealing with file collections
<b>
## Tensorflow library
 It is a library used for machine learnng and deep learning
## String module
 It contains string functions for processing python string
## Pydot
This is an open source graph visualization software

In [2]:
print(tf.__version__)

2.9.1


In [3]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2022-07-14 18:56:16.048911: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory


False

2022-07-14 18:56:16.048948: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-14 18:56:16.048982: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu): /proc/driver/nvidia/version does not exist


In [4]:
# Remove emoji preprocessing code
import emoji

def give_emoji_free_text(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])

    return clean_text

#text = give_emoji_free_text(text)

In [5]:
# Remove url
def rem_url(data_clean):
    p=re.compile(r'\<http.+?\>', re.DOTALL)
    return re.sub(p, '',data_clean)

In [6]:
# Remove punctuations
from string import punctuation
type(punctuation)
my_punctuation = punctuation.replace("'", "")
my_punctuation
def rem_punct(text):
    return text.translate(str.maketrans("", "", my_punctuation))

In [56]:
# Load non_UTF8 data file
#Check the character encoding type
data_path="train.csv"
import chardet
with open(data_path, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result
#From output below, encoding is windows-1252

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}

In [57]:
# Load dataset
# data=pd.read_csv("train.csv")
data = pd.read_csv("train.csv",encoding="utf-8")


In [58]:
data.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [59]:
# Check null values
data.isnull().sum()
#Drop nall
data=data.dropna()
# Check after dropping null
data.isnull().sum()

textID           0
text             0
selected_text    0
sentiment        0
dtype: int64

In [60]:
data.shape

(27480, 4)

In [61]:
data.dropna(axis=0,how="any",inplace=True)
# Remove wors less than 2
data["word_count"]=data["text"].apply(lambda x:len(str(x)))
mask=data["word_count"]>2

In [62]:
data=data[mask]
data["text"]=(data["text"].
      apply(rem_url).
      apply(give_emoji_free_text).
      apply(rem_punct)
     )

In [63]:
data[["sentiment"]].value_counts()

sentiment
neutral      11117
positive      8582
negative      7781
dtype: int64

In [64]:
data.shape

(27480, 5)

In [65]:
longest_sent=data["word_count"].max()
longest_sent

141

In [66]:
# # Remove emoji
# def delete_emoji(data):
#     emoji= re.compile("["
#             u"\U0001F600-\U0001F64F"  # emoticons
#             u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#             u"\U0001F680-\U0001F6FF"  # transport & map symbols
#             u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#             u"\U0001F1F2-\U0001F1F4"  # Macau flag
#             u"\U0001F1E6-\U0001F1FF"  # flags
#             u"\U0001F600-\U0001F64F"
#             u"\U00002702-\U000027B0"
#             u"\U000024C2-\U0001F251"
#             u"\U0001f926-\U0001f937"
#             u"\U0001F1F2"
#             u"\U0001F1F4"
#             u"\U0001F620"
#             u"\u200d"
#             u"\u2640-\u2642"
#             "]+", flags=re.UNICODE)

#     return emoji.sub(r'',data)


In [67]:
test= pd.read_csv("test.csv",encoding="utf-8")
test.shape

(3534, 3)

In [68]:
#Preprocessing test data
# Check null values
test.isnull().sum()
#Drop nall
test=data.dropna()
# Check after dropping null
test.isnull().sum()
test.dropna(axis=0,how="any",inplace=True)
# Remove wors less than 2
test["word_count"]=test["text"].apply(lambda x:len(str(x)))
mask=test["word_count"]>2


test=test[mask]
test["text"]=(test["text"].
      apply(rem_url).
      apply(give_emoji_free_text).
      apply(rem_punct)
     )

In [69]:
test["sentiment"].value_counts()

neutral     11112
positive     8582
negative     7780
Name: sentiment, dtype: int64

In [70]:
longest_sent_test=test["word_count"].max()
longest_sent_test

137

In [71]:

#Tokenizing.
# Splitting sentense in words 
words=3000
token=Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',num_words=words,lower=True,oov_token=None)
token.fit_on_texts(data["text"].tolist())


In [72]:
print(token.texts_to_sequences(["I`d have responded, if I were going"]))

[[1, 469, 16, 68, 1, 119, 45]]


In [73]:
X=data["text"].tolist()
y=data["sentiment"].tolist()

In [74]:
# Output is the seuence of indexes

In [75]:
# Spliting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42,stratify=y)


In [76]:
#Padding add 0 to sebtenses with less words in the set
X_train=np.array(token.texts_to_sequences(X_train),dtype="object")
X_train=np.array(token.texts_to_sequences(X_test),dtype="object")
test=np.array(token.texts_to_sequences(test["text"].tolist()),dtype="object")

                 

In [77]:
X_train[0]

[10, 559, 5, 237, 1872, 47, 14, 758, 3, 24, 351, 16, 170, 1702, 2, 8]

In [78]:
#Padding
#X_train=pad_sequences(X_train,padding="post",maxlen=145)
X_test=pad_sequences(X_test,padding="post",maxlen=145)
#test=pad_sequences(test,padding="post",maxlen=145)

ValueError: invalid literal for int() with base 10: 'IN PAIN my big toe got stomped on during the hokey cokeu its throbbing anyone have any suggestions to heal it'

In [79]:
X_train[0]

[10, 559, 5, 237, 1872, 47, 14, 758, 3, 24, 351, 16, 170, 1702, 2, 8]

In [30]:
# Label encoding
le=LabelEncoder()
y_label=le.fit_transform(y)
print(y_label[0:5])
y_label=np.array(tf.keras.utils.to_categorical(y_label))
y_label[0:5]

[1 0 0 0 0]


array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)